In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import math
import pandas as pd
import pickle
import time
from tqdm.notebook import tnrange
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

In [2]:
def LoadData(PATH,size=100,channels=3) :
    Labels = []
    X_train = []
    Y_train = []
    for i in os.listdir(PATH) :
        print(f'\r\t {i} \t' ,end="")
        Labels.append(i)
        for j in os.listdir(PATH + '/' + i) :
            try :
                img = cv2.imread(PATH + '/' + i + '/' + j)
                img = cv2.resize(img , (size,size))
                #img = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
                X_train.append(img)
                Y_train.append(Labels.index(i))
            except :
                pass
        print(f'\r{i} - {len(set(Y_train))}')

    X_train , Y_train = np.array(X_train)/255. , np.array(Y_train)
    return X_train , Y_train , Labels

In [3]:
loc = 'LoadedData_New.obj'

In [4]:
f = open(loc , 'rb')
X_train,Y_train, Labels = pickle.load(f)
#X_train = X_train.astype(np.double)

In [5]:
fileName = 'Model_Batch_1.1.0_New.h5'

In [6]:
def callback(obj,fileName) :
    print('CallBack : ',fileName,'Updated')
    f = open(fileName, 'wb') 
    pickle.dump(obj, f)

In [7]:
print(f'X_train shape : {X_train.shape}, Y_train shape : {Y_train.shape}')

X_train shape : (3587, 100, 100, 3), Y_train shape : (3587,)


In [8]:
set(Y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [9]:
Labels

['100',
 '30',
 '60',
 'Green',
 'Left',
 'No',
 'Red',
 'Right',
 'Stop',
 'st_Left',
 'st_Right',
 'Yellow']

### Activations

In [10]:
class ReLU :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'ReLU'
    
    def feed(self,X) :
        self.input = X
        self.input_shape=X.shape
        self.output = np.maximum(0,X)
        self.output_shape = self.input_shape
        return self.output
    
    def feed_back(self,Z,grad_output,lr) :
        print(Z.shape,grad_output.shape)
        grad = Z
        grad[Z < 0] = 0
        #print(grad.shape,grad_output.shape)
        return grad_output*grad
    
class TanH :
    def __init__(self) :
        self.__type__ = 'activation'
        self.__Name__ = 'TanH'
    def feed(self,X) :
        self.input = X
        self.input_shape = X.shape
        self.output = (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))
        self.output_shape = self.output.shape
        return self.output
        
    def feed_back(self,Z,grad_output,lr) :
        return 1-np.power(self.output,2)
        
class Softmax :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'Softmax'
    
    def feed(self,X) :
        self.input = X
        e_x = np.exp(X - np.max(X))
        self.output = e_x/np.sum(e_x)
        return self.output
    
    def grad_feed(self,X) :
        e_x = np.exp(X)
        out = (e_x/e_x.sum()) - (np.power(e_x,2)/np.power(e_x.sum(),2))
        #out = e_x*((1/np.sum(e_x)) - (e_x/np.power(e_x.sum(),2)))
        return out
    
    def feed_back(self,Z,grad_output,lr) :
        e_x = np.exp(Z - np.max(Z))
        out = e_x/np.sum(e_x)
        softmax = np.reshape(out, (1, -1))
        grad_output = np.reshape(grad_output, (1, -1))
        grad = (softmax * np.identity(softmax.size) - softmax.transpose() @ softmax)
        return grad_output*grad
    
class Sigmoid :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'Sigmoid'
    
    def feed(self,X) :
        self.input = X
        self.output = 1/(1+np.exp(-X))
        return self.output
    
    def grad_feed(self,X) :
        out = 1/(1+np.exp(-X))
        return out*(1-out)
    
    def feed_back(self,Z,grad_output,lr) :
        out = 1/(1+np.exp(-Z))
        grad = out*((1-out)**2)
        return grad_output*grad

### Layers

In [11]:
class Conv2D :
    
    def __init__ (self,N_F,K_S,A_F,input_shape,STRIDES=1,pad=0) :
        self.__Name__ = 'Conv2D'
        self.__type__ = 'conv'
        self.N_F = N_F
        self.K_S = K_S
        self.STRIDES = STRIDES
        self.A_F = A_F
        self.pad = pad
        if (len(input_shape) == 2) :
            self.input_shape = input_shape+(1,)
        else :
            self.input_shape = input_shape
        self.weights = np.random.normal(size=(self.N_F,self.input_shape[-1],self.K_S,self.K_S))
        self.W = int((self.input_shape[0]-self.K_S+2*self.pad)/self.STRIDES) + 1
        self.H = int((self.input_shape[1]-self.K_S+2*self.pad)/self.STRIDES) + 1
        self.D = self.N_F
        self.bias = np.random.randint(-1,2,(self.N_F))
        self.output_shape = (self.W,self.H,self.D)

    def feed(self,X) :
        output = np.zeros((self.W,self.H,self.D))
#         print(X.shape,output.shape)
        for i in range(0,self.W,self.STRIDES) :
            for j in range(0,self.H,self.STRIDES) :
                cur_reg = X[i:i+self.K_S,j:j+self.K_S].T*self.weights
                output[i,j] = np.sum(cur_reg,axis=(1,2,3)) + self.bias
        
        return output
    
    def plotImg(self,X=None) :
        if X is None :
            X = self.output
        Filter_SIZE = int(X.shape[-1]**(1/2))
        _, axs = plt.subplots(Filter_SIZE,Filter_SIZE, figsize=(8,8))
        axs = axs.flatten()
        for i , ax in enumerate(axs) :
            img = X[:,:,i]
            ax.axis('off')
            ax.imshow(img)
        plt.show()
        
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)

# size=100
# conv = Conv2D(16,3,'ReLU',input_shape=(size,size))
# conv.Summary()
# img = cv2.imread('D:/Data/TestData/70_sign.jfif')
# plt.imshow(img)
# plt.show()
# img = cv2.resize(img,(size,size))
# img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)/255.
# output = conv.feed(img)
# conv.Summary()
# conv.plotImg(output)
# conv = Conv2D(32,3,'ReLU',input_shape=output.shape)
# output = conv.feed(output)
# conv.plotImg(output)
# conv.Summary()
# conv = Conv2D(64,3,'ReLU',input_shape=output.shape)
# output = conv.feed(output)
# conv.plotImg(output)
# conv.Summary()

In [12]:
class MaxPool2D :
    
    def __init__ (self,K_S,input_shape,STRIDES=2,pad=0) :
        self.__Name__ = 'MaxPool2D'
        self.__type__ = 'pool'
        self.K_S = K_S
        self.STRIDES = STRIDES
        self.A_F = None
        self.input_shape = input_shape
        self.output_shape = (int((self.input_shape[0]-self.K_S+1)/self.STRIDES),int((self.input_shape[0]-self.K_S+1)/self.STRIDES),self.input_shape[-1])
        
    def feed(self,X) :
        self.input = X
        W,H,D = self.output_shape
        self.output = np.zeros((W,H,D))
        w , h = 0 , 0
        for i in range(0,self.input_shape[0]-self.K_S,self.STRIDES) :
            for j in range(0,self.input_shape[1]-self.K_S,self.STRIDES) :
                #print(X[i:i+self.K_S,j:j+self.K_S])
                self.output[w,h] = np.max(np.max(X[i:i+self.K_S,j:j+self.K_S].T,axis=1),axis=1)
                # (98,10) , (100,12)
                h += 1
            h = 0
            w += 1
            
        return self.output
    
    def plotImg(self,X=None) :
        if X is None :
            X = self.output
        Filter_SIZE = int(X.shape[-1]**(1/2))
        _, axs = plt.subplots(Filter_SIZE,Filter_SIZE, figsize=(8,8))
        axs = axs.flatten()
        for i , ax in enumerate(axs) :
            img = X[:,:,i]
            ax.axis('off')
            ax.imshow(img)
        plt.show()
        
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)
        
# maxpool = MaxPool2D(3,input_shape=conv.output_shape)
# maxpool.Summary()
# output = maxpool.feed(conv.output)
# maxpool.plotImg(output)

In [13]:
class Flatten :
    
    """
        Flatten class is used to convert data into single dimension
    """
    
    def __init__ (self,input_shape=None,output_shape=None,__Name__='Flatten') : ### Constructor called when we create object 
        self.__Name__ = __Name__ ### Defining __Name__ variable with Flatten
        self.__type__ = 'flat' ### Defining __type__ variable
        self.input_shape = input_shape
        self.A_F = None
        re = 1
        if output_shape is None :
            for i in input_shape :
                re *= i
            self.output_shape = re
        else :
            self.output_shape = output_shape
        
    def feed(self,X) : ### feed function is used to transforms data into single dimension
        #self.input = X
        self.output = X.ravel() ### ravel is used to convert data into single dimensoin or flattens data
        return self.output
    
    def feed_back(self,X,output_error) :
        return X.reshape(self.input_shape)
    
    def predict(self,Y) :
        return Y.ravel()
    
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)

In [14]:
class Dense :
    
    def __init__ (self,input_shape,N_F,A_F=None,wt=None,bias=None,output_shape=None,__Name__='Dense') :
        
        """
            Wt and Bias range [-2.4/No of nodes , 2.4/No of nodes] Proposed by range
        """
        self.__Name__ = __Name__
        self.__type__ = 'dense'
        self.input_shape = input_shape
        self.N_F = N_F
        self.A_F = A_F
        if output_shape is None :
            self.output_shape = N_F
        else :
            self.output_shape = output_shape
        if wt is None :
            self.weights = np.random.uniform(-2.4/self.N_F,2.4/self.N_F,(self.input_shape, self.output_shape))
        else :
            self.weights = wt
        if bias is None :
            self.bias = np.random.uniform(-2.4/ self.N_F,2.4/ self.N_F,(1, self.output_shape))
            #self.bias = np.zeros((1, self.output_shape))
        else :
            self.bias = bias
        self.Batch_W = []
        self.Batch_B = []
        self.v_ = np.zeros((self.input_shape, self.output_shape))
        self.s_ = np.zeros((self.input_shape, self.output_shape))
        self.vv_ = np.zeros((self.output_shape))
        self.ss_ = np.zeros((self.output_shape))
        
    def feed(self,X) :
        self.input = X
        self.output = np.matmul(X,self.weights) + self.bias
        return self.output

    def feed_back(self, Z , output_error, learning_rate=[1e-03],decay=1e-03,opt='sgd',b1=.9,b2=.999):
        input_error = np.dot(output_error,self.weights.T)
        weights , bias = None , None
        weights_error = np.dot(Z.T, output_error)
        lr = learning_rate[np.random.randint(len(learning_rate))]
        if opt.lower() == 'gd' :
                weights = self.weights - lr * weights_error
                bias = self.bias - lr * output_error
        elif opt.lower() == 'sgd' :
                weights = self.weights - lr * weights_error - lr * (decay/self.N_F) * input_error
                bias = self.bias - lr * (decay/self.N_F) * output_error
        elif opt.lower() == 'rmsprop' :
                self.v_ = b1 * self.v_ + .1 * np.power(weights_error,2)
                weights = self.weights - lr * weights_error/(np.sqrt(np.maximum(self.v_,0))+decay)
                bias = self.bias - lr * output_error
        elif opt.lower() == 'adam' :
                self.v_ = b1 * self.v_ + (1-b1) * weights_error
                self.s_ = b2 * self.s_ + (1-b2) * (weights_error * weights_error)
                self.V_ = self.v_ / (1-b1)
                self.S_ = self.s_ / (1-b2)
                weights = self.weights - lr * (self.V_ / (np.sqrt(np.maximum(self.S_,0))+ decay))
                self.vv_ = b1 * self.vv_ + (1-b1) * output_error
                self.ss_ = b2 * self.ss_ + (1-b2) * (output_error * output_error)
#                 self.VV_ = self.vv_ / (1-b1)
#                 self.SS_ = self.ss_ / (1-b2)
#                 bias = self.bias - lr * (self.VV_ / (np.sqrt(np.maximum(self.SS_,0))+ decay))
                bias = self.bias - lr * output_error
        return (weights , bias , input_error)
    
    def predict(self,Y) :
        return np.matmul(Y,self.weights) + self.bias
    
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)

In [15]:
class Dropout :
    def __init__(self,input_shape=None,drop=.5,__Name__ = 'Dropout',A_F=None) :
        self.__Name__ = __Name__
        self.__type__ = 'drop'
        self.N_F = drop
        self.input_shape=input_shape
        self.output_shape = self.input_shape
        self.weights = np.random.binomial(1, drop, self.input_shape) / drop
        self.bias = np.zeros(self.input_shape)
        self.A_F = A_F
        self.Batch_W = []
        self.Batch_B = []
        
    def feed(self,X) :
        return X*self.weights
    
    def feed_back(self, Z , output_error, learning_rate=[1e-03],decay=1e-03,opt='sgd') :
        return (self.weights,self.bias , Z*output_error/self.N_F)
    
    def predict(self,Y) :
        return Y*self.weights
    
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)

In [16]:
def PlotAcc(model) : ### Accuracy Plot
    plt.plot(range(model.ep),model.acc,c='b',label='acc')
    plt.plot(range(model.ep),model.val_acc,c='r',label='val_acc')
    plt.legend()
    plt.show()

def PlotError(model) : ### Error Plot
    plt.plot(range(model.ep),model.error,c='b',label='loss')
    plt.plot(range(model.ep),model.val_error,c='r',label='val_loss')
    plt.legend()
    plt.show()

In [17]:
class Sequential :
    
    """
        Sequential is a class which is used to stack layers of model and to fit , predict , predicting classes of our given i/p
    """
    
    def __init__ (self) :
        self.Layers = []
        self.input_shape = None
        self.Activations = []
        self.acc = []
        self.val_acc = []
        self.error = []
        self.val_error = []
        self.id = 1
        self.ep = 1
        
    def add(self,Layer) :
        boo = False
        for layer in self.Layers :
            if Layer.__type__ == layer.__type__ :
                boo = True
                if '_' not in Layer.__Name__ :
                    Layer.__Name__ += '_'+str(self.id)
                name,k = Layer.__Name__.split('_')
                Layer.__Name__ = name+'_'+str(int(k)+1)
        if not boo :
            if '_' not in Layer.__Name__ :
                Layer.__Name__ += '_'+str(self.id)
        
        self.Layers.append(Layer)
        if Layer.__type__ != 'activation' :
            if self.input_shape is None :
                self.input_shape = Layer.input_shape
            self.output_shape = Layer.output_shape
        if Layer.A_F is not None :
            if Layer.A_F.lower() == 'softmax' :
                self.Activations.append(Softmax())
            elif Layer.A_F.lower() == 'sigmoid' :
                self.Activations.append(Sigmoid())
            elif Layer.A_F.lower() == 'tanh' :
                self.Activations.append(TanH())
            else :
                self.Activations.append(ReLU())
        else :
            self.Activations.append(None)
    def ShuffleData(self,X,Y) :
        order = np.random.randint(0,len(Y),(len(Y)))
        #print(order)
        for i in range(len(Y)-1) :
            X[order[i]] , X[order[i+1]] = X[order[i+1]] , X[order[i]]
            Y[order[i]] , Y[order[i+1]] = Y[order[i+1]] , Y[order[i]]
        return (X , Y)
    
    def feed(self,X) :
        res = []
        for i in tnrange(len(X)) :
            out = X[i]
            for layer,act in zip(self.Layers,self.Activations) :
                out = layer.feed(out)
                if act is not None :
                    out = act.feed(out)
            res.append(out)
            
        return np.array(res)
    
    def SplitData(self,X,Y,split) :
        input_data , output_data , val_input_data , val_output_data = None , None , None , None
        N = int(len(Y) * (1-split))
        while True :
            X , Y = self.ShuffleData(X,Y)
            if set(Y[:N]) != set(Y[N:]) :
                X ,Y = self.ShuffleData(X,Y)
            else :
                input_data , output_data = X[:N] , Y[:N]
                val_input_data , val_output_data = X[N:] , Y[N:]
                break
        return (input_data , output_data , val_input_data , val_output_data)
        
    def compile(self,optimizer='sgd',loss='cross_entropy',metrics=['acc']) :
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        
    def one_hot_encode(self,l) :
        Labels = np.zeros(self.Layers[-1].output_shape)
        Labels[l] = 1
#         for i,label in enumerate(labels) :
#             Labels[i][label] = 1
        return Labels

    def trainModel(self , b , X , Y , lr,decay) : ### X and Y -> batch size of x and batch size of y and training model batch wise
        error = None
        nam = [i.__Name__ for i in self.Layers]
        ind = 0
        for i in range(len(nam)) :
            if nam[i][:3] == 'Fla' :
                ind = i
                break
        for x,y in zip(X,Y) :
            # Training model
            output = x
            loss = None
            
            """
                Forward Feeding
            """
            
            L_INPUTS , L_OUTPUTS , A_INPUTS , A_OUTPUTS = [] , [] , [] , []
            
#             print('*Forward')
            
            for layer , activation in zip(self.Layers , self.Activations) :
                L_INPUTS.append(output)
                output = layer.feed(output)
                L_OUTPUTS.append(output)
                if activation is not None :
                    A_INPUTS.append(output)
                    output = activation.feed(output)
                    A_OUTPUTS.append(output)
            
            """
                Loss Calculation
            """
#             print('*Loss')
            
            if self.loss == 'cross_entropy' :
                loss = self.cat_crossentropy(A_OUTPUTS[-1],y)
                if math.isnan(loss) :
                    return None
                grad_activation = self.Activations[-1].grad_feed(L_OUTPUTS[-1])
                out_err = self.grad_crossentropy(L_OUTPUTS[-1],Y)*grad_activation
            
            """
                Backward Feeding
            """
            
#             print('*Backward')
            try :
                for i in range(1,len(self.Layers)-1) :
                    nam = self.Layers[-i].__Name__.lower()
                    if nam[:3] != 'fla'  :
#                         print(nam,'*1')
                        if nam[:3] != 'dro' :
                            na = self.Activations[-i].__Name__
                            if na != 'Softmax' :
                                out_err = self.Activations[-i].feed_back(A_INPUTS[-i],out_err,lr)
                        w , b_ , out_err = self.Layers[-i].feed_back(L_INPUTS[-i],out_err,lr,decay,opt=self.optimizer)
#                         print(nam,'*2')
                        self.Layers[-i].Batch_W.append(w)
                        self.Layers[-i].Batch_B.append(b_)
                    else :
                        break
                    
            except Exception as e :
                    print(e)
                    pass
#             print('*Done')
            error = np.mean(loss)
            print('\r Loss : %f ' % (error) , end="")
            
        for i in range(1,len(self.Layers)-1) :
            if self.Layers[-i].__Name__[0] != 'F' :
                Batch_W = np.array(self.Layers[-i].Batch_W)
                Batch_B = np.array(self.Layers[-i].Batch_B)
                self.Layers[-i].weights = np.mean(self.Layers[-i].Batch_W , axis = 0)
                self.Layers[-i].bias = np.mean(self.Layers[-i].Batch_B)
                self.Layers[-i].Batch_W = []
                self.Layers[-i].Batch_B = []
                
        return error
            
    def fit(self,train_data,valid_data=None,validation_split=.1,epochs=10,lr=1e-02,decay=1e-03,batch_size=8,model_updation_epoch=2,PlotView=10) :
        self.epochs = epochs
        self.batch_size = batch_size
        input_data , output_data = None , None
        val_input_data , val_target_data = None , None
        if train_data is None :
            raise ValueError('Training Data Required')
        else :
            input_data = train_data[0]
            output_data = train_data[1]
        N = len(input_data)
        
        if valid_data is None :
            input_data , output_data , val_input_data , val_output_data = self.SplitData(input_data , output_data , validation_split)
        else :
            val_input_data , val_output_data = valid_data[0] , valid_data[1]
        print('\nModel Fitting\n')
        
        N = len(output_data)
        
        for ep in range(epochs) :
            model.ep = ep + 1
            start_ep = time.time()
            error = 0
            acc = 0
            print(f'\nepoch : {ep+1}/{epochs}')
            
            for b,batch in enumerate(tnrange(0,N-batch_size+1,batch_size,desc='batch')) :
                #print(b,batch)
                X , Y = input_data[batch:batch+batch_size] , output_data[batch : batch+batch_size]
                error = self.trainModel(b , X , Y , lr , decay)
                while error is None :
                    model.ep = ep
                    #error = self.trainModel(b , X , Y , lr)
                    callback(model,fileName)
                    return self
            
            accuracy = sum([y == np.argmax(model.predict(x)) for x, y in zip(input_data, output_data)]) / N
            self.acc.append(accuracy)
            ac , loss = [] , []
            for x, y in zip(val_input_data, val_output_data) :
                out = model.predict(x)
                if y == np.argmax(out) :
                    ac.append(1)
                else :
                    ac.append(0)
                loss.append(self.cat_crossentropy(out,y))
            ac = np.mean(ac)
            loss = np.mean(loss)
                
            #val_accuracy = sum([y == np.argmax(model.predict(x)) for x, y in zip(val_input_data, val_output_data)]) / len(val_input_data)
            #self.val_acc.append(val_accuracy)
            self.val_acc.append(ac)
            print(' acc=%f , loss=%f , val_acc=%f , val_loss=%f' % (accuracy , error , ac , loss))
            
            if ep > 3 :
                if np.min(model.error) > error and np.min(model.val_error) > loss:
                    callback(model,fileName)
            else :
                callback(model,fileName)
                
            self.error.append(error)
            self.val_error.append(loss)
            if not (ep+1)%PlotView :
                PlotAcc(self)
                PlotError(self)
            
            #end_ep = time.time()
            
            #print(f'Time Taken for epoch {ep+1} : {end_ep-start_ep}s')
            
        return self
    
    def mse(self,y_true, y_pred):
        return np.mean(np.power(y_true - y_pred, 2))
    
    def mse_prime(self,y_true, y_pred):
        return 2 * (y_pred - y_true) / y_pred.size
    
    def transfer_derivative(self,output):
        return output * (1.0 - output)
    
    def binary_crossentropy(self,pred,Truth) :
        GroundTruth = np.zeros(self.No_of_outs)
        GroundTruth[Truth] = 1
        return -np.mean(GroundTruth*np.log(pred)+(1-GroundTruth)*np.log(1-pred))
    
    def binary_grad_crossentropy(self,pred,Truth) :
        GroundTruth = np.zeros(self.No_of_outs)
        GroundTruth[Truth] = 1
        return -((GroundTruth/pred)-((1-GroundTruth)/(1-pred)))
    
    def cat_crossentropy(self,pred,Truth) :
        Truth = self.one_hot_encode(Truth)
        return np.dot((Truth/(pred)),self.Activations[-1].grad_feed(self.Layers[-1].output)[0])

    def grad_cat_crossentropy(self,pred,Truth) :
        Truth = self.one_hot_encode(Truth)
        a = Truth/pred
        b = self.Activations[-1].grad_feed(self.Layers[-1].output)[0]
        return np.dot((Truth/pred),self.Activations[-1].grad_feed(self.Layers[-1].output)[0])
    
    def crossentropy(self,logits,reference_answers):
        return - logits[0][reference_answers] + np.log(np.sum(np.exp(logits),axis=-1))
    
    def grad_crossentropy(self,logits,reference_answers):
        ones_for_answers = np.zeros_like(logits)
        ones_for_answers[np.arange(len(logits)),reference_answers] = 1
        softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
        return (- ones_for_answers + softmax) / logits.shape[0]
    
    def showImg(self,X) :
        plt.imshow(X)
        plt.show()
    
    def predict(self,X):
        outputs = []
        if X.shape == model.input_shape :
            output = X
            for layer , activation in zip(self.Layers,self.Activations) :
                output = layer.predict(output)
                if activation is not None :
                    output = activation.feed(output)
            outputs.append(output)
        else :
            for output in X :
                for layer , activation in zip(self.Layers,self.Activations) :
                    output = layer.predict(output)
                    if activation is not None :
                        output = activation.feed(output)
                outputs.append(output)
        return np.array(outputs)
    
    def pred_class(self,X) :
        classes = []
        if X.shape == model.input_shape :
            output = self.predict(X)
            return np.argmax(output)
        else :
            for output in X :
                output = self.predict(output)
                classes.append(np.argmax(output))
            return np.array(classes)
    
    def Summary(self) :
        print('='*60)
        print('Model Summary')
        print('_'*60)
        print('Layers',' '*(20-len('Layers')),'Input Shape',' '*(20-len('Input Shape')),'Output Shape',' '*(20-len('Output Shape')))
        print('='*60)
        for Layer in self.Layers :
            if Layer.__type__ != 'activation' :
                Layer.Summary()
                print('_'*60)
        print('='*60)

In [18]:
### feeding data using Conv and Maxpool
prev_model = Sequential()
prev_model.add(Conv2D(8,3,input_shape=X_train[0].shape,A_F='ReLU'))
prev_model.add(MaxPool2D(3,input_shape=prev_model.output_shape))
prev_model.add(Dropout(input_shape=prev_model.output_shape))
prev_model.add(Conv2D(16,5,input_shape=prev_model.output_shape,A_F='ReLU'))
prev_model.add(MaxPool2D(5,input_shape=prev_model.output_shape))
prev_model.Summary()
X_train = prev_model.feed(X_train)

Model Summary
____________________________________________________________
Layers                Input Shape           Output Shape         
Conv2D_1              (100, 100, 3)         (98, 98, 8)
____________________________________________________________
MaxPool2D_1           (98, 98, 8)           (48, 48, 8)
____________________________________________________________
Dropout_1             (48, 48, 8)           (48, 48, 8)
____________________________________________________________
Conv2D_2              (48, 48, 8)           (44, 44, 16)
____________________________________________________________
MaxPool2D_2           (44, 44, 16)          (20, 20, 16)
____________________________________________________________


In [19]:
loc = 'ConvData.obj'
Data = (X_train)
f = open(loc , 'wb')
pickle.dump(Data , f)

In [20]:
loc = 'ConvData.obj'
X_train = pickle.load(open(loc , 'rb'))

EOFError: Ran out of input

In [ ]:
model = Sequential()
#model.add(Flatten(input_shape=model.output_shape))
model.add(Flatten(input_shape=Data[0][0].shape))
model.add(Dense(input_shape=model.output_shape,N_F=100,A_F='ReLU')) ### Input Layer
model.add(Dropout(input_shape=model.output_shape))
model.add(Dense(input_shape=model.output_shape,N_F=len(set(Y_train)),A_F='Softmax')) ### Output Layer
model.compile(optimizer='adam',loss='cross_entropy',metrics=['acc'])

model.Summary()

In [ ]:
model.fit(train_data=(X_train,Y_train),
          epochs=100,
          lr=[0.001],
          decay=1e-08,
          batch_size=2,
          PlotView=5)

In [ ]:
def SaveModel(obj,fileName) :
    f = open(fileName, 'wb') 
    pickle.dump(obj, f)
    
def loadModel(fileName) :
    f = open(fileName, 'rb') 
    model = pickle.load(f)
    return model

In [ ]:
PlotAcc(model)
PlotError(model)

In [ ]:
model = loadModel(fileName)

In [ ]:
model.pred_class(X_train[0])

In [ ]:
c = 1001
plt.imshow(X_train[c])
plt.show()
Labels[model.pred_class(X_train[c])]

In [ ]:
c = 0
for i in range(len(X_train)) :
    #print(model.pred_class(X_train[i]),Y_train[i])
    if model.pred_class(X_train[i]) == Y_train[i] :
        c += 1
print(c*100/len(X_train))

In [ ]:
img = X_train[1]
print(model.predict(img))
#print(Labels[model.pred_class(img)])
plt.imshow(img)
plt.show()

In [ ]:
img = X_train[100]
print(model.predict(img))
print(Labels[model.pred_class(img)])
plt.imshow(img)
plt.show()

In [ ]:
X = np.random.randint(0,2,(3))
Y = np.random.randint(0,2,(3))
X , Y

In [ ]:
X@Y

In [ ]:
X*Y

In [ ]:
np.matmul(X,Y)

In [ ]:
X*Y